In [17]:
import csv
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import mediapipe as mp
import cv2


GPT 4: 
FACEMESH_TESSELATION: This connection type is used to draw the tesselation of the face. It outlines the overall structure and shape of the face.

FACEMESH_CONTOURS: These connections are used to outline the contours of the face, such as the jawline, lips, and eyes.

FACEMESH_IRISES: This type specifically focuses on the landmarks around the irises, allowing for detailed rendering of the eye region.

When using MediaPipe Face Mesh in Python, these


https://mediapipe.readthedocs.io/en/latest/solutions/face_mesh.html

Format Data

In [18]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_face_mesh = mp.solutions.face_mesh
def extract_coordinates():
    rows = []
    #creating empty file in folder, I added the start_time in the name of the csv file, so that if a symbol appears many times in a video, it will still be created in two different csv files, just that they will have different starting times
    # csv_file = f"demo.csv"
    # if os.path.exists(csv_file):
    #     return 



# Setup CSV File for the videos
# 21 right hand landmarks, 21 left hand landmarks, 33 pose landmarks
    # num_coords = 21 + 21 + 33
    # landmarks = []
    # for val in range(1, num_coords+1):
    #     landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
    # print("Initialized an empty landmarks of size:", len(landmarks))

    # with open(csv_file, mode='w', newline='') as f:
    #     csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    #     csv_writer.writerow(landmarks)
    
    #size of the videos: 
    # (640, 480)
    size = (640, 480)
    # # that mp4 doesn't work :)
    # final_vid = cv2.VideoWriter('test.mp4', cv2.VideoWriter.fourcc(*'MP4V'), 10, size)
    final_vid = cv2.VideoWriter('test.avi',  
                         cv2.VideoWriter_fourcc(*'MJPG'), 
                         10, size)

#created graph with body landmark connections.
    bodyConnect = [(0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5), (5, 6), (6, 8), 
    # (10, 24), (9, 23), these are the the middle stuff 
    (12, 14), (14, 20), (11, 13), (13, 19), 
    (24, 26), (26, 28), (23, 25), (25, 27), 
    (12, 11)]

# https://github.com/rcsmit/python_scripts_rcsmit/blob/master/extras/Gal_Gadot_by_Gage_Skidmore_4_5000x5921_annotated_black_letters.jpg
    # faceConnect = [(234, 93), (93, 132), (132, 58), (58, 172), (172, 136), (136, 150), 
    # (150, 149), (149, 176), (176, 148), (148, 152), (152,377), 
    # (377, 400), (400, 378), (378, 379), (379, 365), (365, 397), (397, 367), 
    # (367, 288), (288, 435), (435, 361), (361, 401), (401, 323), (323, 366), (366, 451), #face contours
    # (57, 77), (77, 89), (89, 88), (88, 178), (178, 87), (87, 14), (14, 317), (402, 319), (319, 307), #mouth
    # (64, 59), (59, 44), (44, 1), (1, 457), (457, 294),  #straight nose
    # (1, 4), (4, 5), (5, 195), (195, 197), (197, 6), (6, 168), (168, 8),  #vertical up
    # (190, 222), (222, 224), (224, 124), #left eye
    # (413, 441), (441, 442), (442, 443), (443, 445) #right eye
    # ]
    # faceConnect = []

#working with each video
    # cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture("Stationary_moonwalk-yt.mp4")
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    # Read until video is completed
    else: 
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while(cap.isOpened()):  
                # Capture frame-by-frame
                ret, image = cap.read() 
                if ret == True:
                    # the BGR image to RGB.
                    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
                    # To improve performance, optionally mark the image as not writeable to
                    # pass by reference.
                    image.flags.writeable = False

                    #!results contains all the information about the image, in this case, we are looking at hands
                    results = holistic.process(image)
                    
                    
                    # Draw the hand annotations on the image.
                    image.flags.writeable = True
                    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

                    #creating a black background.
                    image = np.zeros(image.shape , np.uint8)
                    
                    # Display the resulting frame
                    # Right hand
                    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=1),
                                            mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                                            )

                    # Left Hand
                    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(121,22,76), thickness=1, circle_radius=1),
                                            mp_drawing.DrawingSpec(color=(121,44,250), thickness=1, circle_radius=1)
                                            )

                    # Pose Detections
                    mp_drawing.draw_landmarks(image,  results.pose_landmarks, bodyConnect,# mp_holistic.POSE_CONNECTIONS, 
                                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=1),
                                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=1, circle_radius=1)
                                            )
                    
                    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_face_mesh.FACEMESH_TESSELATION, #mp_holistic.FACEMESH_CONTOURS,#mp_face_mesh.FACEMESH_IRISES,  
                    #                         mp_drawing.DrawingSpec(color=(80,22,10), thickness=1, circle_radius=1),
                    #                         mp_drawing.DrawingSpec(color=(80,44,121), thickness=1, circle_radius=1)
                    #                         )


                    cv2.imshow('Frame',image)

                    #writing into the video.
                    final_vid.write(image) 



                    # Press Q on keyboard to  exitx
                    if cv2.waitKey(25) & 0xFF == ord('q'):
                        # frame_width = int(cap.get(3)) 
                        # frame_height = int(cap.get(4)) 
                        
                        # size = (frame_width, frame_height) 
                        # result = cv2.VideoWriter('test.mp4',  
                        #  cv2.VideoWriter_fourcc(*'MP4V'), 
                        #  10, size) 
                        print(size)
                        break
                    # Export coordinates
                    # try:
                    #     # Extract Pose landmarks
                    #     if results.pose_landmarks:
                    #         pose = results.pose_landmarks.landmark
                    #         pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
                    #     else:
                    #         # continue
                    #         pose_row=list(np.array([[0,0,0,0] for i in range(33)]).flatten())
                    #     # Extract hands landmarks
                    #     if results.right_hand_landmarks:
                    #         right_hand = results.right_hand_landmarks.landmark
                    #         right_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in right_hand]).flatten())
                    #     else:
                    #         #If no right hand detected, then it writes 0 to the CSV file
                    #         right_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())
                    #     if results.left_hand_landmarks:
                    #         left_hand = results.left_hand_landmarks.landmark
                    #         left_hand_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in left_hand]).flatten())
                    #     else:
                    #         #If no left hand detected, then it writes 0 to the CSV file
                    #         left_hand_row = list(np.array([[0,0,0,0] for i in range(21)]).flatten())

                    #     # Concate rows
                    #     row = pose_row + right_hand_row + left_hand_row
                    #     rows.append(row)

                        # Export to CSV
                        

                    # except Exception as e:
                    #     print(e)
                    #     break
                         
                else:
                    break

            # When everything done, release the video capture object
            cap.release() 

            #this saves into the video
            final_vid.release()


            # Closes all the frames
            cv2.destroyAllWindows()
            # with open(csv_file, mode='a', newline='') as f:
            #                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            #                 for row in rows:
            #                     csv_writer.writerow(row) 
            # return results

In [19]:
extract_coordinates()

Error opening video stream or file


In [14]:
# !ffmpeg -i  test.avi   test.mp4
# # trimming the video from second 2 to 6
!ffmpeg -ss 22 -to 27 -i test.mp4 -c copy output.mp4

# cropping 150 pixels each side

#this is more narrow
# !ffmpeg -i output.mp4 -vf "crop=in_w-300:in_h:150:0" output3.mp4

#this is more wide
# !ffmpeg -i output.mp4 -vf "crop=in_w-200:in_h:200:0" output2.mp4

#this is more more narrow
# !ffmpeg -i output.mp4 -vf "crop=in_w-350:in_h:100:0" output4.mp4

!ffmpeg -i output.mp4 -vf "crop=in_w-350:in_h:150:0" output4.mp4




ffmpeg version 6.0-6ubuntu1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-2ubuntu1)
  configuration: --prefix=/usr --extra-version=6ubuntu1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librist --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libtwolame --enable-libvidstab --en

In [ ]:
# get size
!ffprobe -v error -select_streams v:0 -show_entries stream=width,height -of csv=s=x:p=0 output4.mp4

#this is the size of my videos.

290x480


In [ ]:
#cropping the youtube video


In [ ]:
class ListNode:
    def __init__(self, value=0, next=None):
        self.value = value
        self.next = next

def reverse_linked_list(lst):
    new_lst = None
    while lst != None:
        temp_list = ListNode(lst.value, new_lst)
        new_lst = temp_list
        lst = lst.next
    return new_lst